In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pickle
import scipy.io
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from MLib.Core.losses import MMD_tf
from MLib.Core.kernels import gaussian_kernel
import h5py
from MLib.Learners.KerasLearners import SimpleCNN
from MLib.DomainAdaptation.DA import find_optimal_rotation_simple

2022-01-14 00:56:49.717580: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-14 00:56:49.717651: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def normalize_usps_img(image, label):
    #Normalizes images: `uint8` -> `float32`.
    float_img =  tf.cast(image, tf.float32)

    # Resize the image to (28,28)
    resized_img = tf.image.resize(float_img, (28,28))

    # Reshape the label to (batch_size, )
    reshaped_label = tf.squeeze(label)

    return resized_img, reshaped_label

def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

In [3]:
# Load the MNIST dataset 
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

2022-01-14 00:56:52.711098: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-14 00:56:52.711155: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-14 00:56:52.711229: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (batch-effects): /proc/driver/nvidia/version does not exist
2022-01-14 00:56:52.711589: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Read the USPS dataset
with h5py.File('./Data/usps.h5', 'r') as hf:
    train = hf.get('train')
    X_tr = train.get('data')[:]
    y_tr = train.get('target')[:]
    test = hf.get('test')
    X_te = test.get('data')[:]
    y_te = test.get('target')[:]

usps_train = np.reshape(X_tr, (X_tr.shape[0], 16, 16, 1))
usps_test = np.reshape(X_te, (X_te.shape[0], 16, 16, 1))

ds_usps_train = tf.data.Dataset.from_tensor_slices((usps_train, y_tr))
ds_usps_train = ds_usps_train.map(normalize_usps_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_usps_train = ds_usps_train.batch(128)
ds_usps_train = ds_usps_train.cache()
ds_usps_train = ds_usps_train.prefetch(tf.data.AUTOTUNE)

ds_usps_test = tf.data.Dataset.from_tensor_slices((usps_test, y_te))
ds_usps_test = ds_usps_test.map(normalize_usps_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_usps_test = ds_usps_test.batch(128)
ds_usps_test = ds_usps_test.cache()
ds_usps_test = ds_usps_test.prefetch(tf.data.AUTOTUNE)

In [5]:
# Load the pretrained network. If needed, you can retrain the network by commenting
# the load function and uncommenting the train one
weights_path = './Model_20_CNN.pkl'
num_units = 20
classifier = SimpleCNN(num_units)
classifier.load_from_pickle(weights_path)
# classifier.train(ds_train, save_name='./New_Model_CNN.pkl')

In [6]:
# Map MNIST into the embeddings
# Get the embeddings of the train set of MNIST
correctly_predicted = 0
num_instances = 0
out_ds_train = np.zeros((1,num_units))
lbl_ds_train = np.zeros((1,))
for batch in ds_train:
    batch_img, batch_lbl = batch

    output, prob = classifier.predict(batch_img)
    prediction = np.argmax(prob, axis=1)

    equal_flag = np.equal(prediction, batch_lbl.numpy())
    correctly_predicted += np.sum(equal_flag)
    num_instances += len(equal_flag)

    out_ds_train = np.vstack([out_ds_train, output])
    lbl_ds_train = np.concatenate([lbl_ds_train, batch_lbl])

out_ds_train = out_ds_train[1:,:]
lbl_ds_train = lbl_ds_train[1:]

# Get the embedding of the test set of MNIST
correctly_predicted = 0
num_instances = 0
out_ds_test = np.zeros((1,num_units))
lbl_ds_test = np.zeros((1,))
for batch in ds_test:
    batch_img, batch_lbl = batch

    output, prob = classifier.predict(batch_img)
    prediction = np.argmax(prob, axis=1)

    equal_flag = np.equal(prediction, batch_lbl.numpy())
    correctly_predicted += np.sum(equal_flag)
    num_instances += len(equal_flag)

    out_ds_test = np.vstack([out_ds_test, output])
    lbl_ds_test = np.concatenate([lbl_ds_test, batch_lbl])

out_ds_test = out_ds_test[1:,:]
lbl_ds_test = lbl_ds_test[1:]

In [7]:
# Map USPS into the embeddings
# Get the accuracy on the train set of USPS
out_usps_train = np.zeros((1,num_units))
lbl_usps_train = np.zeros((1,))
predictions_usps_train = np.zeros((1,))

for batch in ds_usps_train:
    batch_img, batch_lbl = batch

    output, prob = classifier.predict(batch_img)
    prediction = np.argmax(prob, axis=1)

    out_usps_train = np.vstack([out_usps_train, output])
    lbl_usps_train = np.concatenate([lbl_usps_train, batch_lbl])
    predictions_usps_train = np.concatenate([predictions_usps_train, prediction])

out_usps_train = out_usps_train[1:,:]
lbl_usps_train = lbl_usps_train[1:]
predictions_usps_train = predictions_usps_train[1:]

# Get the accuracy on the test set of USPS
out_usps_test = np.zeros((1,num_units))
lbl_usps_test = np.zeros((1,))
predictions_usps_test = np.zeros((1,))

for batch in ds_usps_test:
    batch_img, batch_lbl = batch

    output, prob = classifier.predict(batch_img)
    prediction = np.argmax(prob, axis=1)

    out_usps_test = np.vstack([out_usps_test, output])
    lbl_usps_test = np.concatenate([lbl_usps_test, batch_lbl])
    predictions_usps_test = np.concatenate([predictions_usps_test, prediction])

out_usps_test = out_usps_test[1:,:]
lbl_usps_test = lbl_usps_test[1:]
predictions_usps_test = predictions_usps_test[1:]

In [11]:
# Perform binary classification
classification_original = np.zeros((10,10))
classification_rotated = np.zeros((10,10))
classification_semi = np.zeros((10,10))

rotation_dict = dict()

for i in range(10):
    for j in range(i+1, 10):
        print('\n')
        print('Experiment ' + str(i) + ' vs '+ str(j))
        print('-------------')
        print('\n')
        label_1 = i
        label_2 = j

        # Learn a binary classifier on the embedded space
        # --------------------------------------------------------
        flag_mnist_train = np.logical_or(lbl_ds_train==label_1, lbl_ds_train==label_2)
        flag_mnist_test = np.logical_or(lbl_ds_test==label_1, lbl_ds_test==label_2)

        X_train_mnist = out_ds_train[flag_mnist_train]
        y_train_mnist = lbl_ds_train[flag_mnist_train]

        X_test_mnist = out_ds_test[flag_mnist_test]
        y_test_mnist = lbl_ds_test[flag_mnist_test]

        LR_embed = LogisticRegression(penalty='none')
        LR_embed.fit(X_train_mnist, y_train_mnist)

        # Get the accuracy on the usps
        # -------------------------------------------------------
        flag_usps_test = np.logical_or(lbl_usps_test==label_1, lbl_usps_test==label_2)

        X_test_usps = out_usps_test[flag_usps_test]
        y_test_usps = lbl_usps_test[flag_usps_test]

        accuracy = LR_embed.score(X_test_usps, y_test_usps)
        classification_original[label_1,label_2] = accuracy
        classification_original[label_2,label_1] = accuracy
        

        num_instances_test = np.sum(flag_usps_test)
                  
        per_semisupervised = 0.1

        # Randomly select X% of instances from the train set (labeled)
        # -------------------------------------------------------
        flag_usps_train = np.logical_or(lbl_usps_train==label_1, lbl_usps_train==label_2)
        num_instances_train = np.sum(flag_usps_train)

        random_instances = np.random.permutation(num_instances_train)
        selected_instances = random_instances[0:int(num_instances_test*per_semisupervised)]
        X_labeled = out_usps_train[flag_usps_train]
        y_labeled = lbl_usps_train[flag_usps_train]

        X_semi = X_labeled[selected_instances]
        y_semi = y_labeled[selected_instances]

        # Project into a lower dimensional space the MNIST dataset
        # -------------------------------------------------------
        flag_lbl_train = np.logical_or(lbl_ds_train==label_1, lbl_ds_train==label_2)
        flag_lbl_test = np.logical_or(lbl_ds_test==label_1, lbl_ds_test==label_2)

        X_binary_train = out_ds_train[flag_lbl_train]
        y_binary_train = lbl_ds_train[flag_lbl_train]

        X_binary_test = out_ds_test[flag_lbl_test]
        y_binary_test = lbl_ds_test[flag_lbl_test]

        num_instances = int(X_binary_test.shape[0]/2)
        num_instances = 10000

        X_group_a = X_binary_test[0:num_instances]
        y_label_a = y_binary_test[0:num_instances]

        mu_group_MNIST = np.mean(X_group_a, axis=0)
        cov_group_MNIST = np.cov(X_group_a.T)

        # Use the eigenvalue decomposition of the empirical covariance matrix to estimate the weights
        U_MNIST, S_MNIST, V_MNIST = np.linalg.svd(np.round(cov_group_MNIST, 5))

        S_sqrt_MNIST = np.diag(np.round(np.sqrt(S_MNIST),3))

        num_eigen = np.sum(S_sqrt_MNIST > 0.01)
        num_eigen = 5

        W_hat_MNIST = np.matmul(U_MNIST, S_sqrt_MNIST)[:,0:num_eigen]

        # Project back the data
        inv_cov_MNIST = np.linalg.pinv(np.matmul(W_hat_MNIST.T, W_hat_MNIST))
        mult_term_MNIST = np.matmul(inv_cov_MNIST, W_hat_MNIST.T)
        z_MNIST_test = np.matmul(X_group_a - mu_group_MNIST, mult_term_MNIST.T)
        z_MNIST_train = np.matmul(X_binary_train - mu_group_MNIST, mult_term_MNIST.T)

        # Learn the classifier
        # ----------------------------------------------
        LR = LogisticRegression(penalty='none')
        LR.fit(z_MNIST_train, y_binary_train)

        # Project USPS into a lower dimensional space
        # ---------------------------------------------
        flag_lbl_usps = np.logical_or(lbl_usps_test==label_1, lbl_usps_test==label_2)

        X_usps_test = out_usps_test[flag_lbl_usps]
        y_usps_test = lbl_usps_test[flag_lbl_usps]

        mu_group_usps = np.mean(X_usps_test, axis=0)
        cov_group_usps = np.cov(X_usps_test.T)

        # Use the eigenvalue decomposition of the empirical covariance matrix to estimate the weights
        U_usps, S_usps, V_usps = np.linalg.svd(np.round(cov_group_usps, 5))

        S_sqrt_usps = np.diag(np.round(np.sqrt(S_usps),3))

        W_hat_usps = np.matmul(U_usps, S_sqrt_usps)[:,0:num_eigen]

        inv_cov_usps = np.linalg.pinv(np.matmul(W_hat_usps.T, W_hat_usps))
        mult_term_usps = np.matmul(inv_cov_usps, W_hat_usps.T)
        z_usps = np.matmul(X_usps_test - mu_group_usps, mult_term_usps.T)
        z_semi = np.matmul(X_semi - mu_group_usps, mult_term_usps.T)


        # Find the optimal rotation that aligns the datasets
        initial_R = np.eye(num_eigen)
        c_R_rot, c_MMD_rot = find_optimal_rotation_simple(z_MNIST_test, z_usps, 1, 
            initial_R, 20, max_iter_learning_rate=20)

        c_R_ref, c_MMD_ref = find_optimal_rotation_simple(z_MNIST_test, z_usps, 1, 
            -initial_R, 20, max_iter_learning_rate=20)

        # Rotate the semisupervised dataset
        optimally_rotated_semi = np.matmul(z_semi, c_R_rot.numpy().T)
        optimally_reflected_semi = np.matmul(z_semi, c_R_ref.numpy().T)

        # Compute the error in the different sets
        predictions_rotated = LR.predict(optimally_rotated_semi)
        predictions_reflected = LR.predict(optimally_reflected_semi)
        predictions_identity = LR_embed.predict(X_semi)

        accuracy_rotated = np.mean(np.equal(predictions_rotated, y_semi))
        accuracy_reflected = np.mean(np.equal(predictions_reflected, y_semi))
        accuracy_identity = np.mean(np.equal(predictions_identity, y_semi))

        # Choose the one with the best performance
        rotations = [initial_R, c_R_rot.numpy(), c_R_ref.numpy()]
        accuracies = [accuracy_identity, accuracy_rotated, accuracy_reflected]

        best_index = np.argmax(accuracies)

        # If the rotation/reflection helped, apply it. 
        # Leave it as the original otherwise
        c_R = rotations[best_index]

        if best_index == 0:
            predictions_rotated = LR_embed.predict(X_test_usps)

        else:
            c_R = rotations[best_index]

            # Rotate the dataset
            optimally_rotated_usps = np.matmul(z_usps, c_R.T)

            # Error on the test st
            predictions_rotated = LR.predict(optimally_rotated_usps)       
            

        accuracy = np.mean(np.equal(predictions_rotated, y_usps_test))
        classification_semi[i,j] = accuracy
        classification_semi[j,i] = accuracy

        # Also, compute the error in the unsupervised case
        c_R_unsupervised = c_R_rot.numpy()

        # Rotate the dataset
        optimally_rotated_usps_unsupervised = np.matmul(z_usps, c_R_unsupervised.T)

        # Error on the test st
        predictions_rotated_unsupervised = LR.predict(optimally_rotated_usps_unsupervised)
        accuracy_unsupervised = np.mean(np.equal(predictions_rotated_unsupervised, y_usps_test))
        classification_rotated[i,j] = accuracy_unsupervised
        classification_rotated[j,i] = accuracy_unsupervised



Experiment 0 vs 1
-------------


Initial MMD
tf.Tensor(0.030564924145176448, shape=(), dtype=float64)


Small change in MMD. Stopping.
Initial MMD
tf.Tensor(0.05890691085198463, shape=(), dtype=float64)


0.011619576990231542

Experiment 0 vs 2
-------------


Initial MMD
tf.Tensor(0.015465381582680585, shape=(), dtype=float64)


Initial MMD630369357
tf.Tensor(0.028987183257942917, shape=(), dtype=float64)


Small change in MMD. Stopping.


Experiment 0 vs 3
-------------


Initial MMD
tf.Tensor(0.016371100884399165, shape=(), dtype=float64)


Initial MMD341430875
tf.Tensor(0.021197335395265346, shape=(), dtype=float64)


0.014596064878593107

Experiment 0 vs 4
-------------


Initial MMD
tf.Tensor(0.02012715913984542, shape=(), dtype=float64)


Initial MMD900204887
tf.Tensor(0.024516875714027164, shape=(), dtype=float64)


Small change in MMD. Stopping.


Experiment 0 vs 5
-------------


Initial MMD
tf.Tensor(0.015551933750216726, shape=(), dtype=float64)


Small change in MMD. St

/home/rvega/Envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Initial MMD
tf.Tensor(0.017808956055020808, shape=(), dtype=float64)


Initial MMD510633559
tf.Tensor(0.014722076283275973, shape=(), dtype=float64)


0.0060081171211868185

Experiment 4 vs 8
-------------




/home/rvega/Envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Initial MMD
tf.Tensor(0.010229178175966261, shape=(), dtype=float64)


Small change in MMD. Stopping.
Initial MMD
tf.Tensor(0.013743871265577245, shape=(), dtype=float64)


0.0044466979024009735

Experiment 4 vs 9
-------------


Initial MMD
tf.Tensor(0.01750073504638107, shape=(), dtype=float64)


Initial MMD6505135835
tf.Tensor(0.009457800052979665, shape=(), dtype=float64)


0.0067428791548010825

Experiment 5 vs 6
-------------


Initial MMD
tf.Tensor(0.017101291727462498, shape=(), dtype=float64)


Small change in MMD. Stopping.
Initial MMD
tf.Tensor(0.019269509861269823, shape=(), dtype=float64)


0.012802833674956896

Experiment 5 vs 7
-------------


Initial MMD
tf.Tensor(0.01481880546843213, shape=(), dtype=float64)


Initial MMD7064427315
tf.Tensor(0.020223975522180734, shape=(), dtype=float64)


0.009973658690551896

Experiment 5 vs 8
-------------


Initial MMD
tf.Tensor(0.01363148626165965, shape=(), dtype=float64)


Initial MMD3983647655
tf.Tensor(0.012393850793864847, sh

In [13]:
print('Partial Results Semi')
print(classification_semi - classification_original)

print('Partial Results Unsupervised')
print(classification_rotated - classification_original)

Partial Results Semi
[[ 0.         -0.01605136  0.          0.          0.          0.
   0.          0.          0.          0.06529851]
 [-0.01605136  0.          0.03030303  0.01162791  0.01724138  0.
  -0.02764977  0.19221411  0.          0.28798186]
 [ 0.          0.03030303  0.          0.          0.          0.
   0.          0.05507246  0.          0.04266667]
 [ 0.          0.01162791  0.          0.          0.          0.
   0.          0.          0.          0.47230321]
 [ 0.          0.01724138  0.          0.          0.          0.
   0.          0.          0.13661202  0.36870027]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.37388724]
 [ 0.         -0.02764977  0.          0.          0.          0.
   0.          0.07886435  0.02678571  0.29682997]
 [ 0.          0.19221411  0.05507246  0.          0.          0.
   0.07886435  0.          0.19169329  0.39197531]
 [ 0.          0.          0.          0.  